In [16]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pymc3 as pm

%matplotlib inline

In [ ]:
#Seismic displacement model in one-dimension

#First define main parameters for the model
rho_0 = 1800 #density of the medium in kg/m^3
alpha = 0.31 #Parameter used in Beker's paper to determine primary wave speed pg. 84
beta = 0.25 #Parameter used in Beker's paper to determine primary wave speed pg. 84
nu = 0.25 #Poisson's ratio used in Beker's paper

CP = ((rho0/1000) / alpha)**(1.0 / beta) #Calculating primary wave speed using equation from Beker's paper, pg. 84
CS = np.sqrt((1-2*nu)/(2-2*nu)) * CP #Calculating secondary wave speed using equation from Beker's paper, pg. 84
Root = np.roots([1, -8, 8 * ((2 - nu)/(1 - nu)), -8 / (1 - nu)]) #Calculating the the ratio of the R wave speed to the p wave speed squared using equation found in Harm's and Beker's paper, pg. 20 in Beker's paper
for i in Root:
    if 0<i<1:
        CR = np.sqrt(CS**2 *i) #calculating R wave speed

x_list = np.linspace(0,1000,1000) #wave source located at x=0m, test mass at x=1000m, evaluating seismic displacement every meter
t_list = np.linspace(0,4,100) #lengt of time to evaluate the wave's propagation at each point

#Calculating seismic displacement using equation from Harm's paper "Terrestial Gravity Fluctuations", pg. 31 
def xi_horiz(x, z, t, f, phi): #Where x is the position along the horizontal axis, z is the depth of the point in the medium, t is the time, f is the frequency of the wave, and phi is the phase shift
    omega = 2*np.pi*f #calculating the angular frequency
    ke = omega / CR #Calculating horizontal wave number of the Rayleigh wave
    ks = omega / CS #Calculatin the secondary wave number of the Rayleigh wave
    kp = omega / CP #Calculating the primary wave number of the Rayleigh wave
    q_z_s = np.sqrt(ke**2 - ks**2) #Calculating wave parameter used in Harm's model, pg. 31
    q_z_p = np.sqrt(ke**2 - kp**2) #Calculating wave parameter used in Harm's model, pg. 31
    zeta = np.sqrt(q_z_p / q_z_s) #Calculating wave parameter used in Harm's model, pg. 32
    return (ke * np.exp(q_z_p * z) - zeta * np.exp(q_z_s * z)) * np.sin(ke * x - omega * t + phi)

def xi_vert(x, z, t, f, phi):
    omega = 2*np.pi*f #calculating the angular frequency
    ke = omega / CR #Calculating horizontal wave number of the Rayleigh wave
    ks = omega / CS #Calculatin the secondary wave number of the Rayleigh wave
    kp = omega / CP #Calculating the primary wave number of the Rayleigh wave
    q_z_s = np.sqrt(ke**2 - ks**2) #Calculating wave parameter used in Harm's model, pg. 31
    q_z_p = np.sqrt(ke**2 - kp**2) #Calculating wave parameter used in Harm's model, pg. 31
    zeta = np.sqrt(q_z_p / q_z_s) #Calculating wave parameter used in Harm's model, pg. 32
    return (q_z_p * np.exp(q_z_p * z) - zeta * ke * np.exp(q_z_s * z)) * np.cos(ke * x - omega * t + phi)


fig1 = plt.figure(figsize=(10,10))
ax1 = fig1.add_subplot(111, xlabel = 'x-position (m)', ylabel = 'Horizontal Displacement (m)', title = 'Horizontal Displacement of Rayleigh Wave')
ax1.plot(x_list, xi_horiz(x_list, 0))

fig2 = plt.figure(figsize=(10,10))
ax2 = fig2.add_subplot(111, xlabel = 'x-position (m)', ylabel = 'Vertical Displacement (m)', title = 'Vertical Displacement of Rayleigh Wave')
ax2.plot(x_list, xi_vert(x_list, 0))

In [ ]:
#Model for seismic Newtonian Noise in one-dimension

G = 6.67e-11 #Newton's constant of gravity
rho_0 = 1800 #density of the medium in kg/m^3
V = 100 #volume of each element

xi_field = np.zeros((len(x_list),2)) #creating array of seismic displacement at each x-position

#Calculating the Seismic Newtonian Noise using equation (4.13) from Beker's thesis paper on pg. 92
def Seis_NN(z, t, f, phi, x0): #Where z, t, f, and phi are as defined above and x0 is the position of the test mass
    for i, xn in enumerate(x_list):
        if xn <= C_R * t: #only points the wave has reached will be displaced
            xi_field[i] = G * rho_0 * (1/(x0-xn)**3)*-2*xi_horiz(xn, t)*V, G * rho_0 * (1/(1001-xn)**3)*xi_vert(xn, t)*V 
        else: #points beyond where the wave has reached will not yet be displaced
            xi_field[i]= 0, 0
    seis_NN = np.sum(xi_field, axis = 0) #summing the contributions for each point
    total_seis_NN = np.linalg.norm(seis_NN) #computing the total noise from vertical and horizontal displacements
    return total_seis_NN

T = np.zeros(len(t_list)) #creating array to put seismic NN values into, cannot plug array into function to graph
for j, tn in enumerate(t_list):
    T[j] = Seis_NN(tn)

fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111, xlabel='time (sec)', ylabel='Total Siesmic NN', title='Seismic NN due to propagating Rayleigh wave')
ax1.plot(t_list, T)

